Importing relevant libraries for this part of scraping

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
from tqdm import tqdm

page_number function generate a list of URLs for a range of page numbers.

In [54]:
def page_number(start, end):  
    url = 'https://www.mudah.my/terengganu/properties-for-rent?o='
    list_page = []
    for i in range(start, end+1):
        list_page.append(url+str(i))
    return list_page
#print(page_number(1,2)) - checking function

Scrapes property data from a list of URLs and returns a pandas DataFrame.

In [55]:
def prop_data(list_page):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0"} #google my user agent
    data_list = []
    for url in tqdm(list_page):
        page = requests.get(url=url, headers=headers)
        soup = bs(page.text,"html.parser")
        script = soup.find('script', type='application/ld+json')
        if script is not None:
            script = script.string
            data = json.loads(script)
    
            for item in data[2]['itemListElement']:
                prop_name = item['item'].get('name', '')
                prop_type = item['item'].get('@type', '')
                prop_size = item['item'].get('floorSize', '')
                no_of_room = item['item'].get('numberOfRooms', '')
                prop_link = item['item'].get('url', '')
                monthly_rent = item['item'].get('price', '')
                address = item['item']['address'].get('addressLocality', '')

                house_dict = {
                    'Link': prop_link,
                    'Name': prop_name,
                    'Property Type': prop_type,
                    'No of Rooms' :no_of_room,
                    'Property Size (sq. ft)': prop_size,
                    'Monthly Rent': monthly_rent,
                    'Property Address': address
                }
                data_list.append(house_dict)

    df = pd.DataFrame(data_list)
    return df

list_page = page_number(1,15)
df = prop_data(list_page)  
print(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:35<00:00,  2.35s/it]

                                                  Link  \
0    https://www.mudah.my/kedai-jln-sultan-sulaiman...   
1    https://www.mudah.my/kedai-jln-sultan-zainal-a...   
2    https://www.mudah.my/rumah-untuk-disewa-di-ker...   
3    https://www.mudah.my/lot-3111-jalan-besar-kg-n...   
4    https://www.mudah.my/rumah-untuk-disewa-kg-nyi...   
..                                                 ...   
396  https://www.mudah.my/rumah+Sewa+kijal-10539544...   
397  https://www.mudah.my/Single+Room+Kerteh-104125...   
398  https://www.mudah.my/Bilik+Sewa+SINGLE+Tok+Jem...   
399  https://www.mudah.my/Rumah+Sewa+Bukit+Kuang-10...   
400  https://www.mudah.my/Rumah+berdekatan+Politekn...   

                                                  Name  \
0            Kedai Jln Sultan Sulaiman K. Trg Utk Sewa   
1       Kedai Jln Sultan Zainal Abidin K. Trg Utk Sewa   
2    Rumah Untuk Disewa Di Kerteh (Berdekatan Balai...   
3          Lot 3111, Jalan Besar Kg Nyior Paka, Dungun   
4            

Save the Dataframe into an excel file.

In [56]:
df.to_csv('14.TerengganuTest11p.csv', index=False)